In [1]:
import pandas as pd
import os
import re
import pysyge
import pypyodbc

==== Заполнение БД ====

Укажите путь к новой базе в "dbname" 

In [56]:
from win32com.client import Dispatch
try:
    dbname = r'C:\Users\rfrep\OneDrive\Рабочий стол\NewDB.accdb'
    accApp = Dispatch("Access.Application")
    dbEngine = accApp.DBEngine
    workspace = dbEngine.Workspaces(0)

    dbLangGeneral = ';LANGID=0x0409;CP=1252;COUNTRY=0'
    newdb = workspace.CreateDatabase(dbname, dbLangGeneral, 64)

except Exception as e:
    print(e)

finally:
    accApp.DoCmd.CloseDatabase
    accApp.Quit
    newdb = None
    workspace = None
    dbEngine = None
    accApp = None
    



(-2147352567, 'Ошибка.', (0, 'DAO.Workspace', 'База данных уже существует.', 'jeterr40.chm', 5003204, -2146825084), None)


Укажите путь к базе в "conn_str"

In [2]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\rfrep\OneDrive\Рабочий стол\NewDB.accdb;'
    )
cnxn = pypyodbc.connect(conn_str)
crsr = cnxn.cursor()


In [55]:
#cnxn.close()

In [3]:
df = pd.read_csv('Category.csv',sep=';')

In [4]:
crsr.execute("CREATE TABLE Category (ID_Category  INTEGER  PRIMARY KEY,Category CHAR(20) Not Null unique);")
cnxn.commit()

In [6]:
for index ,row in df.iterrows():
    sql = "insert into Category(ID_Category, Category) values (?, ?)"
    params = [row.ID_Category,row.Category]
    crsr.execute(sql, params)
    cnxn.commit()

In [19]:
df = pd.read_csv('Location.csv',sep=';')

In [18]:
crsr.execute("CREATE TABLE Location (IP  CHAR(16)  PRIMARY KEY,Country CHAR(40) Not Null);")
cnxn.commit()

In [20]:
for index ,row in df.iterrows():
    sql = "insert into Location(IP, Country) values (?, ?)"
    params = [row.IP,row.Country]
    crsr.execute(sql, params)
    cnxn.commit()

In [5]:
df = pd.read_csv('Product.csv',sep=';')

In [6]:
crsr.execute("CREATE TABLE Product (ID_Product  INTEGER  PRIMARY KEY,Product CHAR(20) Not Null,Category INTEGER Not Null,FOREIGN KEY (Category) REFERENCES Category(ID_Category));")
cnxn.commit()

In [7]:
for index ,row in df.iterrows():
    sql = "insert into Product(ID_Product, Product, Category) values (?, ?, ?)"
    params = [row.ID_Product,row.Product,row.Category]
    crsr.execute(sql, params)
    cnxn.commit()

In [29]:
df = pd.read_csv('Buy.csv',sep=';')

In [30]:
list =  []
for i in df.User_id:
    list.append(str(i))
df.User_id = list

In [31]:
crsr.execute("CREATE TABLE Buy (Cart_id  INTEGER  PRIMARY KEY,\"Date\" DATE Not Null,\"Time\" Time Not Null,IP CHAR(16) Not Null,User_id CHAR(255),Success_pay Bit Not Null, FOREIGN KEY (IP) REFERENCES Location(IP));")
cnxn.commit()

In [32]:
for index ,row in df.iterrows():
    sql = "insert into Buy(Cart_id, \"Date\", \"Time\",User_id, IP,  Success_pay) values (?, ?, ?, ?, ?, ?)"
    params = [row.Cart_id, row.Date, row.Time, row.User_id, row.IP,  row.Success_pay]
    crsr.execute(sql, params)
    cnxn.commit()

In [33]:
df = pd.read_csv('Basket.csv',sep=';')

In [42]:
crsr.execute("CREATE TABLE Basket (ID_Basket AUTOINCREMENT PRIMARY KEY , \"Date\" DATE Not Null, \"Time\" Time Not Null, IP CHAR(16) Not Null, Goods INTEGER Not Null, Amount INTEGER Not Null,Cart_id INTEGER Not Null,FOREIGN KEY (IP) REFERENCES Location(IP),FOREIGN KEY (Goods) REFERENCES Product(ID_Product));")
cnxn.commit()

In [43]:
for index ,row in df.iterrows():
    sql = "insert into Basket( \"Date\", \"Time\", IP, Goods, Amount, Cart_id) values ( ?, ?, ?, ?, ?, ?)"
    params = [ row.Date, row.Time, row.IP, row.Goods, row.Amount, row.Cart_id]
    crsr.execute(sql, params)
    cnxn.commit()

In [44]:
df = pd.read_csv('Visit.csv',sep=';')

In [52]:
crsr.execute("CREATE TABLE Visit (ID_Visit INTEGER  PRIMARY KEY,\"Date\" DATE Not Null,\"Time\" Time Not Null,IP CHAR(16) Not Null,Product CHAR(16),Times_of_Day CHAR(16) Not Null,Category INTEGER Not Null,FOREIGN KEY (IP) REFERENCES Location(IP),FOREIGN KEY (Category) REFERENCES Category(ID_Category));")
cnxn.commit()

In [53]:
for index ,row in df.iterrows():
    sql = "insert into Visit( ID_Visit,\"Date\", \"Time\", IP, Product, Times_of_Day, Category) values ( ?, ?, ?, ?, ?, ?, ?)"
    params = [ row.ID_Visit, row.Date, row.Time, row.IP, row.Product, row.Times_of_Day, row.Category]
    crsr.execute(sql, params)
    cnxn.commit()

==== Работа с БД ====

Укажите путь к базе в "conn_str"

In [2]:
conn_str = (
    r'DRIVER={Microsoft Access Driver (*.mdb, *.accdb)};'
    r'DBQ=C:\Users\rfrep\Documents\Интерсвязь\DB_Logs.accdb;'
    )
cnxn = pypyodbc.connect(conn_str)
crsr = cnxn.cursor()

№1 Посетители из какой страны совершают больше всего действий на сайте?

In [3]:
country = []
count = []
crsr.execute("SELECT Location.Country, count(Location.Country) FROM Location, Visit, Category WHERE Location.IP = Visit.IP AND Visit.Category = Category.ID_Category Group By Location.Country ORDER BY 2 DESC") 
for row in crsr.fetchall():
    country.append(row[0])
    count.append(row[1])

In [4]:
df = pd.DataFrame({'Country':country, 'Count':count})

№2 Посетители из какой страны чаще всего интересуются товарами из определенных категорий?

In [5]:
category = 'caviar'
crsr.execute("SELECT Location.Country, count(Location.Country) FROM Location, Visit, Category WHERE Location.IP = Visit.IP AND Visit.Category = Category.ID_Category AND Category.Category = (?) Group By Location.Country ORDER BY 2 DESC", [(category)])
for row in crsr.fetchall():
    print (row)

('United States', 1338)
('Not found', 607)
('China', 275)
('Japan', 136)
('United Kingdom', 121)
('South Korea', 95)
('Australia', 80)
('Germany', 74)
('Switzerland', 60)
('France', 58)
('Canada', 55)
('Belgium', 46)
('Brazil', 44)
('Russia', 43)
('Thailand', 41)
('Italia', 39)
('Netherlands', 38)
('Malaysia', 36)
('South Africa', 25)
('Taiwan', 25)
('Morocco', 25)
('Mexico', 24)
('India', 22)
('Egypt', 21)
('Saudi Arabia', 21)
('Israel', 18)
('Indonesia', 18)
('Ireland', 16)
('Romania', 16)
('Spain', 15)
('Sweden', 15)
('Argentina', 14)
('Mauritius', 13)
('Denmark', 13)
('Finland', 13)
('Poland', 12)
('Norway', 11)
('Nigeria', 11)
('Austria', 10)
('Ukraine', 10)
('United Arab Emirates', 10)
('Colombia', 8)
('Chile', 7)
('Ghana', 7)
('Kuwait', 7)
('Lithuania', 7)
('Luxembourg', 7)
('Venezuela', 7)
('New Zealand', 6)
('Latvia', 6)
('Tunisia', 6)
('Iran', 6)
('Philippines', 6)
('Peru', 6)
('Croatia', 6)
('Bosnia and Herzegovina', 5)
('Slovakia', 5)
('Kazakhstan', 5)
('Hungary', 5)
('Malt

№3 В какое время суток чаще всего просматривают определенную категорию товаров?

In [6]:
category = 'caviar'
crsr.execute("SELECT Visit.Times_of_Day, count(Visit.Times_of_Day) FROM Visit, Category WHERE Visit.Category = Category.ID_Category AND Category.Category = (?) Group By Visit.Times_of_Day ORDER BY 2 DESC", [(category)])
for row in crsr.fetchall():
    print (row)

('Morning', 1268)
('Evening', 1001)
('Day', 754)
('Night', 692)


№4 Какая нагрузка (число запросов) на сайт за астрономический час?

In [7]:
t = '2:30:00'

spl = re.split(':',t)
tt = str(int(spl[0])+1)+':'+spl[1]+':'+spl[2]

dt = '01.08.2018'
crsr.execute("select  count(*) from Visit where Visit.Time Between [?] and [?] and Visit.Date = [?];",[(t),(tt),(dt)]) 
for row in crsr.fetchall():
    print (row[0])

68


№5 Товары из какой категории чаще всего покупают совместно с товаром из заданной категории?

In [8]:
crsr.execute("SELECT Category.Category, Count(Category.Category) FROM Product, Category, Basket WHERE (((Basket.Cart_id) In (select Basket.Cart_id From Buy, Basket Where Buy.Cart_id = Basket.Cart_id and Basket.Goods in  (select Product.ID_Product From Product , Category Where Product.Category = Category.ID_Category and Category.Category = (?)))) AND ((Basket.Goods)=[Product].[ID_Product]) AND ((Product.Category)=[Category].[ID_Category]) AND ((Category.Category)<>([?]))) GROUP BY Category.Category ORDER BY 2 DESC;", [(category),(category)])
for row in crsr.fetchall():
    print (row)

('frozen_fish', 41)
('semi_manufactures', 40)
('fresh_fish', 38)
('canned_food', 38)


№6 Сколько брошенных (не оплаченных) корзин имеется за определенный период?

In [9]:
date1 = '01.08.2018'
date2 = '02.08.2018'
crsr.execute("SELECT Count(*) FROM Basket WHERE (((Basket.Cart_id) Not In (select Basket.Cart_id From Buy, Basket Where Buy.Cart_id = Basket.Cart_id))) AND Basket.Date Between [?] and [?];",[(date1),(date2)]) 
for row in crsr.fetchall():
    print (row[0])

29


№7 Какое количество пользователей совершали повторные покупки за определенный период?

In [10]:
date1 = '01.08.2018'
date2 = '02.08.2018'
crsr.execute("select count(*) from (SELECT count(Buy.User_id) AS Count_Visit FROM Buy  WHERE Buy.Date Between [?] and [?] Group By Buy.User_id ) AS Unique_User where  Unique_User.Count_Visit > 1 ;",[(date1),(date2)]) 
for row in crsr.fetchall():
    print (row[0])

2
